# Installing Dependecies

In [220]:
install.packages("class")


  There is a binary version available but the source version is later:
      binary source needs_compilation
class 7.3-19 7.3-20              TRUE



installing the source package 'class'

Warning message in install.packages("class"):
"installation of package 'class' had non-zero exit status"

In [1]:
library(class)

# Reading Data

In [2]:
data <- read.csv("hotel_bookings.csv", stringsAsFactors = 0)
dim(data)

[1] 119390     32

# Preprocessing


In [3]:
data$children[is.na(data$children)] = 0

levels(data$country) <- c(levels(data$country), "UNKNOWN") 
data$country[data$country == "NULL"] <- "UNKNOWN"
data$country = factor(data$country)

levels(data$agent) <- c(levels(data$agent), '0')
data$agent[data$agent == "NULL"] <- '0'
data$agent = factor(data$agent)

levels(data$company) <- c(levels(data$company), '0')
data$company[data$company == "NULL"] <- '0'
data$company = factor(data$company)

threshold = 400
outliers = data[data$lead_time > threshold, ]

percentage = nrow(outliers)/nrow(data) * 100
sprintf("Percentage data that has lead time greater than %d days is %.2f%%", threshold, percentage)

data = data[data$lead_time <= threshold, ]

[1] "Percentage data that has lead time greater than 400 days is 1.77%"

# attribute selection

In [21]:
selected_attrs=c("lead_time","arrival_date_week_number" ,"babies" ,"required_car_parking_spaces", "total_of_special_requests","children")
attributes(data)$names

[1] "hotel"                          "is_canceled"                   
 [3] "lead_time"                      "arrival_date_year"             
 [5] "arrival_date_month"             "arrival_date_week_number"      
 [7] "arrival_date_day_of_month"      "stays_in_weekend_nights"       
 [9] "stays_in_week_nights"           "adults"                        
[11] "children"                       "babies"                        
[13] "meal"                           "country"                       
[15] "market_segment"                 "distribution_channel"          
[17] "is_repeated_guest"              "previous_cancellations"        
[19] "previous_bookings_not_canceled" "reserved_room_type"            
[21] "assigned_room_type"             "booking_changes"               
[23] "deposit_type"                   "agent"                         
[25] "company"                        "days_in_waiting_list"          
[27] "customer_type"                  "adr"                           
[29] "required_car_parking_spaces"    "total_of_special_requests"     
[31] "reservation_status"             "reservation_status_date"

# train

In [20]:

set.seed(101) # Set Seed so that same sample can be reproduced in future also
# Now Selecting 75% of data as sample from total 'n' rows of the data  
sample <- sample.int(n = nrow(data), size = floor(.8*nrow(data)), replace = F)
train <- data[sample, ]
# train= train$lead_time
test  <- data[-sample, ]
# test= test

# attrs=c(lead_time,arrival_date_week_number ,babies ,required_car_parking_spaces, total_of_special_requests,children)
prc_train = subset(train, select=selected_attrs)
prc_test = subset(test, select=selected_attrs)
dim(prc_train)
dim(prc_test)

normalize <- function(x) {
return ((x - min(x)) / (max(x) - min(x))) }
prc_train <- as.data.frame(lapply(prc_train, normalize))
prc_test <- as.data.frame(lapply(prc_test, normalize))


prc_test_pred <- knn(train = prc_train, test = prc_test,cl = train$is_canceled, k=10)


[1] 93820     7

[1] 23456     7

ERROR: Error in x - min(x): non-numeric argument to binary operator


# accuracy 

In [22]:
accuracy =  sum(prc_test_pred==test$is_canceled ) / dim(test)[1]
sprintf("knn classifier accuracy is %f", accuracy)

[1] "knn classifier accuracy is 0.757631"